In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from datetime import datetime
import numpy as np
from IPython.display import Image, display

In [2]:
%%bash

JULIA_VERSION="1.4.2"
JULIA_PACKAGES="DataFrames Feather ShiftedArrays RollingFunctions CSV Pipe Plots Flux Images Statistics"

JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
wget -nv $URL -O /tmp/julia.tar.gz
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'"'
done

Installing Julia package DataFrames...
Installing Julia package Feather...
Installing Julia package ShiftedArrays...
Installing Julia package RollingFunctions...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package Plots...
Installing Julia package Flux...
Installing Julia package Images...
Installing Julia package Statistics...


2020-11-07 15:07:21 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz [99093958/99093958] -> "/tmp/julia.tar.gz" [1]
    Cloning default registries into `~/.julia`
    Cloning registry from "https://github.com/JuliaRegistries/General.git"
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed SortingAlgorithms ─────────── v0.3.1
  Installed PooledArrays ──────────────── v0.5.3
  Installed DataAPI ───────────────────── v1.4.0
  Installed TableTraits ───────────────── v1.0.0
  Installed IteratorInterfaceExtensions ─ v1.0.0
  Installed CategoricalArrays ─────────── v0.8.3
  Installed DataStructures ────────────── v0.18.8
  Installed DataFrames ────────────────── v0.21.8
  Installed DataValueInterfaces ───────── v1.0.0
  Installed OrderedCollections ────────── v1.3.2
  Installed Reexport ──────────────────── v0.2.0
  Installed Parsers ───────────────────── v1.0.11
  Installed Missings ───────

In [3]:

import os
os.environ['JULIA_NUM_THREADS'] = "4"

In [4]:
%%time
!pip install --quiet julia
import julia
from julia.api import Julia
julia.install()
jl = Julia(compiled_modules=False)  # cannot use precompiled packages with pyjulia on linux :-(
%load_ext julia.magic


Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.


Initializing Julia interpreter. This may take some time...
CPU times: user 21.1 s, sys: 397 ms, total: 21.5 s
Wall time: 1min 9s


In [6]:
%%julia
directory_path = "../input/dogs-cats-images/dataset/training_set/cats"
directory_files = readdir(directory_path);

In [8]:
%%julia
size(directory_files)[1]

4000

In [10]:
%%julia
directory_images = filter(x -> occursin(r"\.(jpg|png|gif){1}$"i, x), directory_files);

In [17]:
%%julia
using Pkg
Pkg.add("ImageMagick")

  Resolving package versions...
  Installed libpng_jll ────── v1.6.37+6
  Installed Libtiff_jll ───── v4.1.0+2
  Installed ImageMagick_jll ─ v6.9.10-12+3
  Installed Zstd_jll ──────── v1.4.5+2
  Installed JpegTurbo_jll ─── v2.0.1+3
  Installed ImageMagick ───── v1.1.6
#################################################################         12.6%############## 100.0%##O#- #                                                                       ####### 100.0%
######################################################################## 100.0%############### 100.0%##O#- #                                                                       #                                                                          1.5%
######################################################################## 100.0%####################################################################### 100.0%##O=#  #                                                                      ##                                          

In [12]:
%%julia
using Pkg
Pkg.add("ProgressMeter")

   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed ProgressMeter ─ v1.4.0
   Updating `~/.julia/environments/v1.4/Project.toml`
  [92933f4c] + ProgressMeter v1.4.0
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [92933f4c] + ProgressMeter v1.4.0


In [13]:
%%julia
import ProgressMeter

In [18]:
%%julia
using Images
all_images = []
p = ProgressMeter.Progress(size(directory_images)[1])
for image_name in directory_images
    image_path = joinpath(directory_path, image_name)
    image = load(image_path);
#     display(image)
    image = imresize(image, (128,128))
    
#     @info size(image)
#     break
    push!(all_images, image)
    ProgressMeter.next!(p)
end

 + libpng_jll v1.6.37+6
Progress: 100%|█████████████████████████████████████████| Time: 0:00:51


In [19]:
%%julia
using Pkg
Pkg.add("Plots")

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [31]:
%%julia
using Plots

function showBatch(noImages,arr)
    p = plot(axis = nothing,layout = noImages);

    for x in 1:noImages
        plot!(p[x],arr[x])
    
    end
    display(p)
end

<PyCall.jlwrap showBatch>

In [32]:
%%julia
size(all_images)

(4000,)

In [35]:
%%julia
gr()

<PyCall.jlwrap Plots.GRBackend()>

In [36]:
%%julia

showBatch(5, all_images)


In [38]:
%%julia
preprocess(img) = vec(Float64.(convert(Array{Float64},channelview(img));))

<PyCall.jlwrap preprocess>

In [39]:
%%julia

function preprocess2(img)
    return hcat(float.(img)...)
end

<PyCall.jlwrap preprocess2>

In [40]:
%%julia
all_images = preprocess.(all_images);


In [41]:
%%julia
preprocess2(all_images[1])

array([[0.15686275, 0.17647059, 0.16078431, ..., 0.17647059, 0.1372549 ,
        0.10196078]])

In [44]:
%%julia
all_images = preprocess2.(all_images);
typeof(all_images[1])

<PyCall.jlwrap Array{Float64,2}>

In [ ]:
%%julia
using Flux
model = Chain(
    Conv((3,3),3=>16,pad=(1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),16=>32,pad=(1,1),relu),
    MaxPool((2,2)),
    #Conv((3,3),32=>32,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),32=>64,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),64=>128,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),128=>256,pad= (1,1),relu),
    MaxPool((2,2)),
    Conv((3,3),256=>512,pad= (1,1),relu),
    MaxPool((2,2)),
    #x->reshape(x,:,size(x,4)),
    Dense(4096,2),
    softmax,)